In [17]:
import os
import geopandas as gpd

lidar_dir = '../lidar'
counties = sorted(os.listdir(lidar))
dbases = [os.path.join(lidar_dir, c, 'tiles.gpkg') for c in counties 
          if os.path.exists(os.path.join(lidar_dir, c, 'tiles.gpkg'))]
#print(dbases)

In [ ]:
def tindex_fix(dbase):
    tindex = gdb.read_file(dbase)
    tindex[tindex['srs']=='EPSG:4326']
    if len(tindex['srs'].unique()) == 0:
        

In [14]:
import pandas as pd
#counties = ['dewitt', 'mclean', 'piatt']
frames = [gpd.read_file(db) for db in dbases]
tindex = pd.concat(frames, ignore_index=True)
tindex.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 223328 entries, 0 to 223327
Data columns (total 5 columns):
location    223328 non-null object
srs         223328 non-null object
modified    223328 non-null object
created     223328 non-null object
geometry    223328 non-null object
dtypes: object(5)
memory usage: 8.5+ MB


In [16]:
print(sorted(tindex['srs'].unique()))

['+proj=tmerc +lat_0=36.66666666666666 +lon_0=-88.33333333333333 +k=0.9999749999999999 +x_0=300000.0000000001 +y_0=0 +ellps=GRS80 +units=us-ft +vunits=us-ft +no_defs ', '+proj=tmerc +lat_0=36.6666666666667 +lon_0=-88.3333333333333 +k=0.9999749999999999 +x_0=300000.0000000001 +y_0=0 +ellps=GRS80 +units=us-ft +vunits=us-ft +no_defs ', '+proj=tmerc +lat_0=36.6666666666667 +lon_0=-90.1666666666667 +k=0.999941176470588 +x_0=699999.9999999993 +y_0=0 +ellps=GRS80 +units=us-ft +vunits=us-ft +no_defs ', 'EPSG:26916', 'EPSG:26971', 'EPSG:26972', 'EPSG:3443', 'EPSG:3444', 'EPSG:3531', 'EPSG:4326']
